In [1]:
FROM_CACHE = False
GPT2_VERSION = 'gpt2-medium'

In [2]:
import winobias
from experiment import Model
from attention_utils import report_interventions_summary_by_head, report_interventions_summary_by_layer, report_intervention, perform_interventions
from transformers import GPT2Tokenizer
import json

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


## Dev Set Results GPT2-Medium (filtering on odds ratio): 

In [ ]:
fname = 'winobias_data/attention_intervention_results_gpt2medium_filtered.json'
if not FROM_CACHE:
	quantile = 0.25
	model = Model(output_attentions=True, gpt2_version=GPT2_VERSION)
	tokenizer = GPT2Tokenizer.from_pretrained(GPT2_VERSION)
	examples = winobias.load_dev_examples()
	
	df = winobias.analyze(examples, gpt2_version=GPT2_VERSION)
	df_expected = df[df.odds_ratio > 1]
	threshold = df_expected.odds_ratio.quantile(quantile)
	
	filtered_examples = []
	assert len(examples) == len(df)
	for i in range(len(examples)):
		ex = examples[i]
		odds_ratio = df.iloc[i].odds_ratio
		if odds_ratio > threshold:
	
			filtered_examples.append(ex)
	print(f'Num examples with odds ratio > 1: {len(df_expected)} / {len(examples)}')
	
	print(f'Num examples with odds ratio > {threshold:.4f} ({quantile} quantile): {len(filtered_examples)} / {len(examples)}')
			 
	examples = filtered_examples
	
	interventions = [ex.to_intervention(tokenizer) for ex in examples]
	results = perform_interventions(interventions, model)
	with open(fname, 'w') as f:
		json.dump(results, f)
with open(fname) as f:
	results = json.load(f)

Split: DEV, Filtered: False
Loaded 160 pairs. Skipped 38 pairs.


 34%|███▍      | 54/160 [01:11<02:14,  1.27s/it]

### Mean Effect

In [ ]:
report_interventions_summary_by_head(results)

In [ ]:
report_interventions_summary_by_layer(results)

### Examples
	

In [ ]:
report_intervention(results[0])

In [ ]:
report_intervention(results[1])

In [ ]:
report_intervention(results[2])

In [ ]:
report_intervention(results[3])
